The Sport Classification - Supervised Learning

In [40]:
# Load libraries
import os
import cv2
import numpy as np
from imutils import paths
import cv2 as cv

In [41]:
datapath = r"C:\Users\maryam.fatima\Videos\Video_cllass\data\data"
outputmodel = r"C:\Users\maryam.fatima\Videos\Video_cllass\video classification model\videomodel"
outputlabelbinarizer = r"C:\Users\maryam.fatima\Videos\Video_cllass\videoclassbinarizer"
epoch=25

In [42]:
# Importing libraries from sklearn and Keras

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [43]:
Sports_labels = set(["basketball", "fencing", "football", "wwe"])
print("Images is being uploaded.....")

pathToImages = list(paths.list_images(datapath))
data = []
labels = []


for images in pathToImages:
    label = images.split (os.path.sep)[-2]
    if label not in Sports_labels:
        continue
    image = cv.imread(images)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image = cv.resize(image, (224, 224))
    data.append(image)
    labels.append(label)

Images is being uploaded.....


In [44]:
data = np.array(data)
labels = np.array(labels)

# Hot encoded values as 0, 1, 2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [45]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [46]:
traininAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

validationAugmentation = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
traininAugmentation.mean = mean
validationAugmentation.mean = mean

In [47]:
# Loading Keras libraries for building the architecture

from keras.applications import ResNet50

from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout

from keras.models import Model

In [48]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(244, 244, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="Flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable = False

In [49]:
from keras.optimizers import SGD

In [50]:
opt = SGD(lr=0.0001, momentum=0.9, decay=1e-4/epoch)

In [51]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [52]:
History = model.fit(
traininAugmentation.flow(X_train, Y_train, batch_size=32),
steps_per_epoch=len(X_train) // 32,
validation_data=validationAugmentation.flow(X_test, Y_test),
validation_steps=len(X_test) // 32, epochs=epoch
)

Epoch 1/25
60/60 [==============================] - ETA: 0s - loss: 1.7357 - accuracy: 0.3400

KeyboardInterrupt: 

In [ ]:
import pickle

model.save(outputmodel)
lbinarizer = open(r"C:\Users\maryam.fatima\Videos\Video_cllass\data\videoclassification", "wb")
lbinarizer.write(pickel.dumps(lb))
lbinarizer.close()